In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_salary = pd.read_csv("../../../data//salary_potential.csv")
df_out = pd.read_csv("../../../data/National_education_cost/CP4-out-state-tuition-current-d.csv")
df_in = pd.read_csv("../../../data/National_education_cost/CP4-state-tuition-current-d.csv")

import warnings
warnings.filterwarnings("ignore")

df_in.head()

,In Current Dollars,STATE,2007-08,2008-09,2009-10,2010-11,2011-12,2012-13,2013-14,2014-15,...,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,1-Year,5-Year
0,University of Alaska Fairbanks,AK,4496.0,4828.0,5133.0,5316.0,5570.0,5898.0,5988.0,6182.0,...,7184.0,7518.0,8087.0,8538.0,8980.0,9070.0,9882.00,10970.0,0.11,0.36
1,University of Alabama,AL,5700.0,6400.0,7000.0,7900.0,8600.0,9200.0,9450.0,9826.0,...,10470.0,10780.0,10780.0,10780.0,11620.0,11620.0,11900.00,11900.0,0.00,0.10
2,University of Arkansas,AR,6038.0,6400.0,6459.0,6768.0,7174.0,7553.0,7818.0,8210.0,...,8820.0,9062.0,9130.0,9380.0,9380.0,9570.0,9656.00,9748.0,0.01,0.07
3,University of Arizona,AZ,5037.0,5531.0,6855.0,8237.0,9299.0,10035.0,10391.0,10957.0,...,11769.0,11877.0,12487.0,12600.0,12700.0,12716.0,13255.36,13600.0,0.03,0.09
4,University of California: Berkeley,CA,7164.0,7656.0,9749.0,10940.0,12835.0,12874.0,12864.0,12972.0,...,13485.0,13928.0,14184.0,14253.0,14310.0,14433.0,15220.00,15602.0,0.03,0.10


In [2]:
df_out["STATE"] = df_in["STATE"] 
columns = ['In Current Dollars', 'STATE']

for i, j in enumerate(columns):
    # print(i,j)
    desired_position, column_name = i,j
    column_index = df_out.columns.get_loc(column_name)
    df_out.insert(desired_position, column_name, df_out.pop(column_name))
df_out.columns

Index(['In Current Dollars', 'STATE', '2007-08', '2008-09', '2009-10',
       '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16',
       '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
       '2022-23', '2023-24', '1-Year', '5-Year'],
      dtype='object')

In [3]:
def get_data_ready(data: pd.DataFrame):
    data = data.rename({
        "In Current Dollars":"University",
        "STATE":"State_Code"
        }, 
        axis=1, 
        # inplace=True
        ).dropna()
    column_to_drop = data.columns[-7:-2].to_list()
    # print(column_to_drop)
    data.drop(column_to_drop, axis=1, inplace=True)
    # print(data.columns)
    
    data = pd.melt(data.iloc[:, 1:], id_vars=["State_Code", "1-Year", "5-Year"], var_name="Year", value_name="Tuition")
    data = data.sort_values(by=["State_Code", "Year"]).reset_index(drop=True)
    relevant = pd.DataFrame(data.groupby(["State_Code"])["Tuition"].mean(""))
    return relevant




In [4]:
relevant_in = get_data_ready(df_in)
relevant_out = get_data_ready(df_out)
relevant_out


,Tuition
State_Code,
AK,18538.916667
AL,23183.166667
AR,19258.500000
AZ,27554.083333
CA,35647.166667
CO,31592.083333
CT,30466.833333
DE,28110.833333
FL,26467.750000


In [5]:
relevant_salary = df_salary.iloc[:, 1:].groupby(["state_name"]).mean().reset_index()
relevant_salary.rename({
    "state_name":"State"
}, axis=1, inplace=True)
relevant_salary['State'] = relevant_salary['State'].apply(lambda x: x.replace("-", " "))
# relevant_salary

state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

# Add state codes to the melted DataFrame
relevant_salary['State_Code'] = relevant_salary['State'].map(state_codes)
relevant_salary.head()

,State,early_career_pay,mid_career_pay,State_Code
0,Alabama,44992.000000,81592.000000,AL
1,Alaska,55133.333333,97600.000000,AK
2,Arizona,50228.571429,90642.857143,AZ
3,Arkansas,44110.526316,79242.105263,AR
4,California,67232.000000,123976.000000,CA


In [6]:
# # merge data
# df_list = [relevant_salary, relevant_four_year, relevant_two_year]
# merged_df = df_list[0]
# for df in df_list:
#     merged_df = pd.merge(right=merged_df, left=df, on="State")
# merged_df

merged_df = pd.merge(right=relevant_out, left=relevant_in, on="State_Code", suffixes=["-In state", "-Out state"]).reset_index()
df = pd.merge(right=merged_df, left=relevant_salary, on="State_Code")


In [7]:
df.drop(['State_Code'], axis=1, inplace=True)
df.head()

,State,early_career_pay,mid_career_pay,Tuition-In state,Tuition-Out state
0,Alabama,44992.000000,81592.000000,8856.333333,23183.166667
1,Alaska,55133.333333,97600.000000,6083.666667,18538.916667
2,Arizona,50228.571429,90642.857143,9489.833333,27554.083333
3,Arkansas,44110.526316,79242.105263,7662.833333,19258.500000
4,California,67232.000000,123976.000000,11840.166667,35647.166667


In [8]:
# x and y given as array_like objects
import plotly.express as px
pay_list = df.columns[1:3].to_list()
univ_type_list =  df.columns[3:].to_list()
pay_list, univ_type_list


(['early_career_pay', 'mid_career_pay'],
 ['Tuition-In state', 'Tuition-Out state'])

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.4, 0.4],
    row_heights=[0.4, 0.4],
    )

for univ_type in univ_type_list:
    for pay in pay_list:
        fig.add_trace(

            go.Scatter(
            x=df[univ_type],
            y=df[pay],
            mode="markers",
            name= f"{univ_type}-{pay}",
            
            
            ),
            row=univ_type_list.index(univ_type)+1,
            col=pay_list.index(pay)+1,
        )
        


fig.update_layout(title_text="Subplots Example", showlegend=True, height=1000)
# fig.write_html("../../../graphs/html/five/scatter_plot_1.html")

fig.show()

In [11]:
df.head()

,State,early_career_pay,mid_career_pay,Tuition-In state,Tuition-Out state
0,Alabama,44992.000000,81592.000000,8856.333333,23183.166667
1,Alaska,55133.333333,97600.000000,6083.666667,18538.916667
2,Arizona,50228.571429,90642.857143,9489.833333,27554.083333
3,Arkansas,44110.526316,79242.105263,7662.833333,19258.500000
4,California,67232.000000,123976.000000,11840.166667,35647.166667


In [13]:
df.iloc[:,1:].corr().iloc[2:, :2]

,early_career_pay,mid_career_pay
Tuition-In state,0.369521,0.350374
Tuition-Out state,0.378423,0.351633


In [14]:
fig = px.imshow(df.iloc[:,1:].corr().iloc[2:, :2], text_auto=True, aspect="auto", width=1000)
# fig.write_html("../../../graphs/html/five/correlation_1.html")
fig.show()